# Your Title Here

**Name(s)**: (your name(s) here)

**Website Link**: (your website link)

In [36]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

# from dsc80_utils import * # Feel free to uncomment and use this.

## Step 1: Introduction

In [138]:
# TODO
df = pd.read_csv('2023_LoL_esports_match_data_from_OraclesElixir.csv')
df = df[df['datacompleteness'] == 'complete']
new_df = df[['teamname','side','gameid','firsttothreetowers','result','league', 'heralds','hextechs','date','team kpm','minionkills','firsttower', 'void_grubs','inhibitors', 'pentakills', 'firstbloodassist', 'url', 'playoffs', 'champion']].sort_values('firsttothreetowers')
team_df = new_df[df['playername'].isna()]
team_df.isna().sort_values('team kpm')

player_df = new_df[df['playername'].notna()]
# team_df[team_df['teamname'] == 'APOLLO GAMING']
# df[df['playername'].isna()]['minionkills']
# df[df['playername'].isna()]

c:\Users\Zachary\miniforge3\envs\dsc80\lib\site-packages\IPython\core\interactiveshell.py:3508: DtypeWarning:

Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.

C:\Users\Zachary\AppData\Local\Temp\ipykernel_37280\2450392506.py:5: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\Zachary\AppData\Local\Temp\ipykernel_37280\2450392506.py:8: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [38]:
for i in df.columns:
    print(i)

gameid
datacompleteness
url
league
year
split
playoffs
date
game
patch
participantid
side
position
playername
playerid
teamname
teamid
champion
ban1
ban2
ban3
ban4
ban5
pick1
pick2
pick3
pick4
pick5
gamelength
result
kills
deaths
assists
teamkills
teamdeaths
doublekills
triplekills
quadrakills
pentakills
firstblood
firstbloodkill
firstbloodassist
firstbloodvictim
team kpm
ckpm
firstdragon
dragons
opp_dragons
elementaldrakes
opp_elementaldrakes
infernals
mountains
clouds
oceans
chemtechs
hextechs
dragons (type unknown)
elders
opp_elders
firstherald
heralds
opp_heralds
void_grubs
opp_void_grubs
firstbaron
barons
opp_barons
firsttower
towers
opp_towers
firstmidtower
firsttothreetowers
turretplates
opp_turretplates
inhibitors
opp_inhibitors
damagetochampions
dpm
damageshare
damagetakenperminute
damagemitigatedperminute
wardsplaced
wpm
wardskilled
wcpm
controlwardsbought
visionscore
vspm
totalgold
earnedgold
earned gpm
earnedgoldshare
goldspent
gspd
gpr
total cs
minionkills
monsterkills
mon

## Step 2: Data Cleaning and Exploratory Data Analysis

In [39]:
# TODO
# custom_legend_names = {1: 'Wins with firsttothree', 0: 'Losses with firsttothree'}
# fig = px.pie(team_df, values='firsttothreetowers',names='result', title = 'Winrate First to Three Turrets')
# fig.update_layout(legend_title = 'Legend')
# fig.for_each_trace(lambda t: t.update(name=custom_legend_names[t.name]))
# fig.show()


# bivariate
fig = px.histogram(team_df, x = 'minionkills')
fig.show()

fig_2 = px.histogram(team_df, x = 'minionkills')
fig_2.show()

In [72]:
# hold = {1.0: True, 0.0: False, np.NaN: False}

# temp = team_df.copy()

# temp.dropna()

# temp = temp.replace({'firsttower' : hold})

# print(temp['firsttower'])

# fig = px.bar(
#     temp, 
#     x='firsttower', 
#     y='firsttothreetowers', 
#     color='firsttower', 
#     color_discrete_map={True: 'red', False: 'blue'}, 
#     title='First to three with First Tower',
#     opacity= 0.5
# )

# fig.show()


team_df.groupby('firsttothreetowers').mean()

,result,team kpm,minionkills,firsttower
firsttothreetowers,,,,
0.0,0.209221,0.334783,805.232556,0.224739
1.0,0.790732,0.588414,823.283594,0.775210


## Step 3: Assessment of Missingness

In [148]:
# TODO
team_df.columns
# team_df.groupby('inhibitors').sum()
# player_df['firstbloodassist'].sort_values(ascending=False)
# team_df[['url','league']].sort_values('url').tail(30).iloc[0]['url']
# team_df[team_df['url'].isna()]

# team_df[team_df['result'].isna()]
# df['champion'].sort_values()

df['hextechs'].sort_values()
df[df['heralds'].isna()].groupby('gameid').sum()['heralds'].sort_values()
# df[df['heralds'].notna()].groupby('gameid').sum()

gameid
ESPORTSTMNT01_3286841    0.0
ESPORTSTMNT03_3199128    0.0
ESPORTSTMNT03_3199118    0.0
ESPORTSTMNT03_3199117    0.0
ESPORTSTMNT03_3199115    0.0
                        ... 
ESPORTSTMNT02_3169368    0.0
ESPORTSTMNT02_3169385    0.0
ESPORTSTMNT02_3169387    0.0
ESPORTSTMNT02_3169174    0.0
ESPORTSTMNT06_2831008    0.0
Name: heralds, Length: 8827, dtype: float64

## Step 4: Hypothesis Testing

In [42]:
# TODO
td = team_df.copy()

td['won_with_three'] = (td['result'] == 1) * ((td['firsttothreetowers'] == 1))
test = td[['firsttothreetowers','result','won_with_three']]
    
observed = (test['won_with_three'].mean())

stat_list = []

for i in range(0,1000):
    test['shuffled_result'] = np.random.permutation(test['result'])
    test['shuffled_won_with_three'] = (test['shuffled_result'] == 1) * ((test['firsttothreetowers'] == 1))

    stat_list.append((test['shuffled_won_with_three'].mean()))

np.abs(observed - np.array(stat_list))


C:\Users\Zachary\AppData\Local\Temp\ipykernel_37280\1017055682.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Zachary\AppData\Local\Temp\ipykernel_37280\1017055682.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



0.12210206995003568

## Step 5: Framing a Prediction Problem

In [43]:
# TODO
team_df

,teamname,side,gameid,firsttothreetowers,result,league,date,team kpm,minionkills,firsttower
107626,DRX,Blue,ESPORTSTMNT01_3411490,0.0,0,LCK,2023-08-08 09:02:21,0.4585,789.0,1.0
51395,Estral Esports,Red,ESPORTSTMNT01_3336144,0.0,1,LLA,2023-03-30 00:06:55,0.4144,1079.0,0.0
102767,Vivo Keyd Stars,Red,ESPORTSTMNT01_3402856,0.0,0,CBLOL,2023-07-29 18:23:28,0.3484,786.0,0.0
51407,Los Grandes Academy,Red,ESPORTSTMNT01_3336147,0.0,1,CBLOLA,2023-03-30 00:26:17,0.7232,914.0,0.0
51418,Estral Esports,Blue,ESPORTSTMNT01_3337087,0.0,0,LLA,2023-03-30 01:11:19,0.1485,1146.0,0.0
...,...,...,...,...,...,...,...,...,...,...
125879,Bilibili Gaming,Red,10659-10659_game_2,NaN,0,DCup,2023-12-31 10:17:43,0.4557,NaN,NaN
125890,Bilibili Gaming,Blue,10659-10659_game_3,NaN,1,DCup,2023-12-31 11:10:34,0.6377,NaN,NaN
125891,Royal Never Give Up,Red,10659-10659_game_3,NaN,0,DCup,2023-12-31 11:10:34,0.1063,NaN,NaN
125902,Royal Never Give Up,Blue,10659-10659_game_4,NaN,0,DCup,2023-12-31 11:48:06,0.4059,NaN,NaN


## Step 6: Baseline Model

In [44]:
# TODO
team_df

,teamname,side,gameid,firsttothreetowers,result,league,date,team kpm,minionkills,firsttower
107626,DRX,Blue,ESPORTSTMNT01_3411490,0.0,0,LCK,2023-08-08 09:02:21,0.4585,789.0,1.0
51395,Estral Esports,Red,ESPORTSTMNT01_3336144,0.0,1,LLA,2023-03-30 00:06:55,0.4144,1079.0,0.0
102767,Vivo Keyd Stars,Red,ESPORTSTMNT01_3402856,0.0,0,CBLOL,2023-07-29 18:23:28,0.3484,786.0,0.0
51407,Los Grandes Academy,Red,ESPORTSTMNT01_3336147,0.0,1,CBLOLA,2023-03-30 00:26:17,0.7232,914.0,0.0
51418,Estral Esports,Blue,ESPORTSTMNT01_3337087,0.0,0,LLA,2023-03-30 01:11:19,0.1485,1146.0,0.0
...,...,...,...,...,...,...,...,...,...,...
125879,Bilibili Gaming,Red,10659-10659_game_2,NaN,0,DCup,2023-12-31 10:17:43,0.4557,NaN,NaN
125890,Bilibili Gaming,Blue,10659-10659_game_3,NaN,1,DCup,2023-12-31 11:10:34,0.6377,NaN,NaN
125891,Royal Never Give Up,Red,10659-10659_game_3,NaN,0,DCup,2023-12-31 11:10:34,0.1063,NaN,NaN
125902,Royal Never Give Up,Blue,10659-10659_game_4,NaN,0,DCup,2023-12-31 11:48:06,0.4059,NaN,NaN


## Step 7: Final Model

In [45]:
# TODO

## Step 8: Fairness Analysis

In [46]:
# TODO